In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
print(torch.cuda.is_available())
if torch.cuda.is_available() and False:
    print ("cuda in use")
    device = torch.device('cuda') 
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    dtype = torch.cuda.FloatTensor
else:
    print ("cuda not used")
    device = torch.device('cpu')
    torch.set_default_tensor_type('torch.FloatTensor')
    dtype = torch.float

True
cuda not used


In [4]:
#from T. Kohonen '84, p. 183, equation 6.33

#Set alpha to one for one-step orthogonalization.
class Novelty_Filter():
    
    def __init__(self,size,epsilon=1e-4):
        self.O = torch.eye(size, requires_grad=False, dtype=dtype, device=device)
        self.epsilon = epsilon
        return
        
    def addBasis(self,X,cf=1.):
        l = self.novelty(X)
        n = l.norm()
        print("norms",n,X.norm())
        if(cf * n > self.epsilon * X.norm()):
            print("suppressing")
            self.O -= cf * l.ger(l) / n.pow(2)
        return
    
#     def addBasis(self,X,cf=0.):
# #         print("cf",cf)
#         #Memorize this (linear) pattern
#         X = self.novelty(X)
#         n = torch.norm(X)
#         if (n*cf > self.epsilon):
#             self.O -= cf * X.ger(X) / n.pow(2)
#         return
    
    def novelty(self,X):
        return self.O @ X
    
    def project(self,X):
        return X - self.novelty(X)

In [5]:
def selu(x):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * F.elu(x, alpha)

class Xor(nn.Module):
    
    def __init__(self):
        super(Xor, self).__init__() 
        hidden_nodes = 51
        self.l1 = nn.Linear(3,hidden_nodes,bias=False)
        self.l1.weight.data.normal_(0.0, np.sqrt(1./3.))
        self.head = nn.Linear(hidden_nodes, 1,bias=False)
        self.head.weight.data.normal_(0.0, np.sqrt(1./hidden_nodes))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.l1_out = selu(self.l1(inputs))
        self.value = self.head(self.l1_out)
        return self.value
    
    def get_loss(self,target):
        self.loss = F.mse_loss(target,self.value)
        self.smoothed_loss = self.smoothed_loss * 0.95 + self.loss * 0.05
        return self.loss

class Stable_Xor(Xor):
    
    def __init__(self,alpha=0.):
        super(Stable_Xor, self).__init__()
        self.alpha = alpha
        self.eps = 1.e-12
        self.l1_filter = Novelty_Filter(self.l1.in_features)
        self.head_filter = Novelty_Filter(self.head.in_features)
        self.smoothed_loss = 1.e-12
        
    def get_loss(self,target):
        super(Stable_Xor, self).get_loss()
        self.smoothed_loss = (self.smoothed_loss * 0.95 + self.loss * 0.05).detach()
        return self.loss
        
    def get_certainty():
        return self.alpha/torch.exp(abs(self.loss.data*4./self.smoothed_loss))
        
    def do_post_gradient(self):
        self.l1.weight.grad *=  self.l1_filter.novelty(self.inputs) / (self.inputs + self.eps)
        self.head.weight.grad *= self.head_filter.novelty(self.l1_out.data) / (self.l1_out.data + self.eps)
    
    def do_pre_update(self):
        certainty_factor = self.get_certainty()
        print("cfx",certainty_factor)
        self.l1_filter.addBasis(self.inputs.data,certainty_factor)
        self.head_filter.addBasis(self.l1_out.data,certainty_factor)
        
data = torch.tensor([[1,1,1,0],[1,0,0,0],[1,0,1,1],[1,1,0,1]], requires_grad=False, dtype=dtype, device=device)
print("data",data)

data tensor([[1., 1., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [1., 1., 0., 1.]])


In [6]:
    
def showme():
    print("Current Forecasts")
    for j in range(4):
        print("input",data[j][0:3],"output",xor(data[j][0:3]).data,"target",data[j][3:4])
    print("End")
    
xor = Stable_Xor(1.)
xopt = optim.Adam(xor.parameters(), lr=1e-1,weight_decay=0.000001)
iterations=100
stable = True



print("---")
for j in range(iterations):
    xor(data[0][0:3])
    xor.get_loss(data[0][3:4]).backward()
    if stable: xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()

print("input",data[0][0:3],"output",xor(data[0][0:3]).data,"target",data[0][3:4])
    
xor(data[0][0:3])
loss=xor.get_loss(data[0][3:4])
print("loss",loss,xor.value,data[0][3:4])
loss.backward()
if stable: xor.do_post_gradient()
if stable: xor.do_pre_update()
xopt.step()
xopt.zero_grad()

showme()
for j in range(iterations):
    xor(data[1][0:3])
    xor.get_loss(data[1][3:4]).backward()
    if stable: xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()
    
print("input",data[1][0:3],"output",xor(data[1][0:3]).data,"target",data[1][3:4])
xor(data[1][0:3])
loss=xor.get_loss(data[1][3:4])
print("loss",loss,xor.value,data[1][3:4])
loss.backward()
xor.do_post_gradient()
xor.do_pre_update()
xopt.step()
xopt.zero_grad()

showme()
for j in range(iterations):
    xor(data[2][0:3])
    xor.get_loss(data[2][3:4]).backward()
    if stable: xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()
    
print("input",data[2][0:3],"output",xor(data[2][0:3]).data,"target",data[2][3:4])
xor(data[2][0:3])
loss=xor.get_loss(data[2][3:4])
print("loss",loss,xor.value,data[2][3:4])
loss.backward()
if stable: xor.do_post_gradient()
if stable: xor.do_pre_update()
xopt.step()
xopt.zero_grad()
showme()
for j in range(iterations):
    xor(data[3][0:3])
    loss =  xor.get_loss(data[3][3:4])
    loss.backward()
    if stable: xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()
    
print("input",data[3][0:3],"output",xor(data[3][0:3]).data,"target",data[3][3:4])
xor(data[3][0:3])
loss=xor.get_loss(data[3][3:4])
print("loss",loss,xor.value,data[3][3:4])
loss.backward()
if stable: xor.do_post_gradient()
if stable: xor.do_pre_update()
xopt.step()
xopt.zero_grad()
    
showme()

---
input tensor([1., 1., 1.]) output tensor([-0.0004]) target tensor([0.])
loss tensor(1.4983e-07, grad_fn=<MeanBackward0>) tensor([-0.0004], grad_fn=<SqueezeBackward3>) tensor([0.])
cfx tensor(1.0000)
norms tensor(1.7321) tensor(1.7321)
suppressing
norms tensor(3.5217) tensor(3.5217)
suppressing
Current Forecasts
input tensor([1., 1., 1.]) output tensor([0.0057]) target tensor([0.])
input tensor([1., 0., 0.]) output tensor([-0.4531]) target tensor([0.])
input tensor([1., 0., 1.]) output tensor([0.4286]) target tensor([1.])
input tensor([1., 1., 0.]) output tensor([-0.9214]) target tensor([1.])
End
input tensor([1., 0., 0.]) output tensor([-0.0016]) target tensor([0.])
loss tensor(2.5112e-06, grad_fn=<MeanBackward0>) tensor([-0.0016], grad_fn=<SqueezeBackward3>) tensor([0.])
cfx tensor(1.0000)
norms tensor(0.8165) tensor(1.)
suppressing
norms tensor(3.7344) tensor(3.8567)
suppressing
Current Forecasts
input tensor([1., 1., 1.]) output tensor([0.0282]) target tensor([0.])
input tensor(

In [129]:
for i in range(100):
    for j in range(4):
        xor(data[j][0:3])
        loss =  xor.get_loss(data[j][3:4])
        loss.backward()
        xor.do_post_gradient()
        xopt.step()
        xopt.zero_grad()
showme()

Current Forecasts
input tensor([1., 1., 1.]) output tensor([-1.7869]) target tensor([0.])
input tensor([1., 0., 0.]) output tensor([-2.2013]) target tensor([0.])
input tensor([1., 0., 1.]) output tensor([-3.4242]) target tensor([1.])
input tensor([1., 1., 0.]) output tensor([0.1817]) target tensor([1.])
End


In [ ]:
    
xor = Stable_Xor(1.)
xopt = optim.Adam(xor.parameters(), lr=1e-1,weight_decay=0.000001)
iterations=500
print("---")
for j in range(iterations):
    xor(data[0][0:3])
    xor.get_loss(data[0][3:4]).backward()
    xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()

print("input",data[0][0:3],"output",xor(data[0][0:3]).data,"target",data[0][3:4])
print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))
    
xor(data[0][0:3])
loss=xor.get_loss(data[0][3:4])
print("loss",loss,xor.value,data[0][3:4])
loss.backward()
xor.do_post_gradient()
xor.do_pre_update()
xopt.step()
xopt.zero_grad()
print("post novelty of input, is",xor.l1_filter.novelty(xor.inputs))
print("post novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))

print("input",data[1][0:3],"output",xor(data[1][0:3]).data,"target",data[1][3:4])
print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))
for j in range(iterations):
    xor(data[1][0:3])
    xor.get_loss(data[1][3:4]).backward()
    xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()

print("input",data[1][0:3],"output",xor(data[1][0:3]).data,"target",data[1][3:4])
print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
print("post novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))
# print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
# print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))
    
print("final")
for j in range(4):
    print("input",data[j][0:3],"output",xor(data[j][0:3]).data,"target",data[j][3:4])

In [ ]:
xor(data[1][0:3])
loss=xor.get_loss(data[1][3:4])
print("loss",loss,xor.value,data[1][3:4])
loss.backward()
xor.do_post_gradient()
xor.do_pre_update()
xopt.step()
xopt.zero_grad()

for j in range(iterations):
    xor(data[2][0:3])
    xor.get_loss(data[2][3:4]).backward()
    xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()

print("input",data[2][0:3],"output",xor(data[2][0:3]).data,"target",data[2][3:4])
# print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
# print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))

xor(data[2][0:3])
loss=xor.get_loss(data[2][3:4])
print("loss",loss,xor.value,data[2][3:4])
loss.backward()
xor.do_post_gradient()
xor.do_pre_update()
xopt.step()
xopt.zero_grad()

for j in range(iterations):
    xor(data[3][0:3])
    xor.get_loss(data[3][3:4]).backward()
    xor.do_post_gradient()
    xopt.step()
    xopt.zero_grad()

print("input",data[3][0:3],"output",xor(data[3][0:3]).data,"target",data[3][3:4])
# print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
# print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))

xor(data[3][0:3])
loss=xor.get_loss(data[3][3:4])
print("loss",loss,xor.value,data[3][3:4])
    
print("final")
for j in range(4):
    print("input",data[j][0:3],"output",xor(data[j][0:3]).data,"target",data[j][3:4])
#     print("novelty of input, is",xor.l1_filter.novelty(xor.inputs))
#     print("novelty of hidden, is",xor.head_filter.novelty(xor.l1_out))
    
print(xor.l1_filter.O)
print(xor.head_filter.O[0:5])

In [ ]:
    
f = Novelty_Filter(3)
X = torch.tensor([7,.1,11], requires_grad=False, dtype=dtype, device=device)
X2 = torch.tensor([7,-3,.1], requires_grad=False, dtype=dtype, device=device)
v = torch.ones([1], requires_grad=False, dtype=dtype, device=device)
print(X)
alpha = 0.2
for i in range(10):
    certainty_factor = alpha/torch.exp(abs(i*.1*v))
    print(certainty_factor)
    f.addBasis(X,certainty_factor)
    print(f.novelty(X),f.novelty(X2))
    print(torch.norm(X),torch.norm(f.novelty(X)))

In [ ]:
# xor = Xor()
# xopt = optim.Adam(xor.parameters(), lr=3e-2,weight_decay=0.00001)

# print(data)
# for i in range(1000):
#     for j in range(4):
#         xor(data[j][0:2])
#         xor.get_loss(data[j][2:3]).backward()
#     xopt.step()
#     xopt.zero_grad()

# for j in range(4):
#     print(xor(data[j][0:2]).data,data[j][2:3])
    
# xor = Xor()
# xopt = optim.Adam(xor.parameters(), lr=3e-2,weight_decay=0.00001)

# print("---")
# for i in range(1000):
#     for j in range(4):
#         xor(data[j][0:2])
#         xor.get_loss(data[j][2:3]).backward()
#         xopt.step()
#         xopt.zero_grad()
        
# for j in range(4):
#     print(xor(data[j][0:2]).data,data[j][2:3])


In [ ]:
    
#from T. Kohonen '84, p. 119, equation 4.63, p. 122, equation 4.68
class Adaptive_Novelty(Fast_Novelty):
    
    def __init__(self,size,alpha=0.95,gamma=1e-3):
        super(Adaptive_Novelty,self).__init__(size,alpha)
        self.alpha = alpha
        self.gamma = gamma
        
    def addBasis(self,X):
        X = X.data
        #Slowly learn to remember this (linear) pattern
        O2 = self.O.pow(2)
        self.O -= self.alpha*O2*X.ger(X)*O2
        #Very gradually forget everything we've learned
        if self.gamma > 0.:
            self.O += self.gamma * (self.O - O2)
        return

In [ ]:
O = torch.eye(3, requires_grad=False, dtype=dtype, device=device)
a = 0.003
g = 0.01
X = torch.tensor([7,0.17,11], requires_grad=False, dtype=dtype, device=device).t()
X2 = torch.tensor([7,-3,.1], requires_grad=False, dtype=dtype, device=device).t()
O2 = np.square(O)
print(a*(O2*X*X.t()*O2))
print(a*(O2*(X*X.t())*O2))
print(a*O2*X.ger(X)*O2)
for i in range(1000):
    n = O @ X
    O2 = np.square(O)
    O -= a*O2*X.ger(X)*O - g * (O - O2)
    
#     O = O - a*O2*X.ger(X)*O + g * (O - O2)
print(X,X2)
print(O @ X, O@X2)
print(X - O@X, X2 - O@X2)

print('---')
# O = torch.eye(3, requires_grad=False, dtype=dtype, device=device)
for i in range(1000):
    n = O @ X2
    O2 = np.square(O)
    O -= a*O2*X2.ger(X2)*O - g * (O - O2)
print(X,X2)
print(O @ X, O@X2)
print(X - O@X, X2 - O@X2)

In [ ]:
X = torch.tensor([7,0.17,11], requires_grad=False, dtype=dtype, device=device).t()
X2 = torch.tensor([7,-3,.1], requires_grad=False, dtype=dtype, device=device).t()
filter = Adaptive_Novelty(3,alpha=0.01,gamma=5e-2)
filter.addBasis(X)
print(filter.novelty(X2),filter.novelty(filter.novelty(filter.novelty(X2))))
filter.addBasis(X2)

filter2 = Adaptive_Novelty(3,alpha=0.01,gamma=0.)
print(filter2.novelty(X2))
filter2.addBasis(X)
print(filter2.novelty(X2))
filter2.addBasis(filter2.novelty(X2))
print(filter.O,filter2.O)
print(filter.project(X2),filter2.project(X2))

In [ ]:
X = torch.tensor([1,0,0], requires_grad=False, dtype=dtype, device=device).t()
X2 = torch.tensor([0,1,1], requires_grad=False, dtype=dtype, device=device).t()
O = torch.eye(3, requires_grad=False, dtype=torch.float)
a = 7e-3
print(O @ X,O@X2)
d=O@X
d2=O@X2
for i in range(100000):
    n = O @ X
    O2 = O.pow(2)
    O = O - a*O2*X.ger(X)*O2
print(O @ X, O@X2)
print(X - O@X, X2 - O@X2)
print(O@X/d)
print(O@X2/d2)
# for i in range(10000):
#     n = O @ X2
#     O2 = O.pow(2)
#     O = O - a*O2*X2.ger(X2)*O2
# print(O @ X, O@X2)
# print(X - O@X, X2 - O@X2)



In [ ]:
ortho = Orthogonalizer(3)
print(ortho.novelty(X),ortho.novelty(X2))
ortho.addBasis(X2)
print(ortho.novelty(X),ortho.novelty(X2))
print(ortho.project(X),ortho.project(X2))

Y = torch.stack([X,X,X]).numpy()
Y2 = torch.stack([X2,X2,X2]).numpy()
print(Y,Y2)
Q,r = np.linalg.qr(Y)
print(Q@Q.T)
print(ortho.O)
print(Q @ Q.T @ Y)#Projection from Q
print(Q @ Q.T @ Y2)#Projection from Q

In [59]:
def showme2():
    print('----------')
    print(data[0][0:3],ortho.project(data[0][0:3]))
    print(data[1][0:3],ortho.project(data[1][0:3]))
    print(data[2][0:3],ortho.project(data[2][0:3]))
    print(data[3][0:3],ortho.project(data[3][0:3]))
    print('==========')

data = torch.tensor([[1,1,1,0],[1,0,0,0],[1,0,1,1],[1,1,0,1]], requires_grad=False, dtype=dtype, device=device)
print("data",data)

ortho = Novelty_Filter(3)
print(ortho.O)
print("adding",data[0][0:3])
ortho.addBasis(data[0][0:3])
print(ortho.O)
showme2()
print("adding",data[1][0:3])
ortho.addBasis(data[1][0:3])
print(ortho.O)
showme2()
print("adding",data[2][0:3])
ortho.addBasis(data[2][0:3])
print(ortho.O)
showme2()
print("adding",data[3][0:3])
ortho.addBasis(data[3][0:3])
print(ortho.O)

showme2()

data tensor([[1., 1., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [1., 1., 0., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
adding tensor([1., 1., 1.])
norms tensor(1.7321) tensor(1.7321)
suppressing
tensor([[ 0.6667, -0.3333, -0.3333],
        [-0.3333,  0.6667, -0.3333],
        [-0.3333, -0.3333,  0.6667]])
----------
tensor([1., 1., 1.]) tensor([1., 1., 1.])
tensor([1., 0., 0.]) tensor([0.3333, 0.3333, 0.3333])
tensor([1., 0., 1.]) tensor([0.6667, 0.6667, 0.6667])
tensor([1., 1., 0.]) tensor([0.6667, 0.6667, 0.6667])
adding tensor([1., 0., 0.])
norms tensor(0.8165) tensor(1.)
suppressing
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.5000, -0.5000],
        [ 0.0000, -0.5000,  0.5000]])
----------
tensor([1., 1., 1.]) tensor([1., 1., 1.])
tensor([1., 0., 0.]) tensor([1., 0., 0.])
tensor([1., 0., 1.]) tensor([1.0000, 0.5000, 0.5000])
tensor([1., 1., 0.]) tensor([1.0000, 0.5000, 0.5000])
adding tensor([1., 0., 1.])
norms

In [ ]:

I = torch.eye(3, requires_grad=False, dtype=dtype, device=device)
V1 = torch.tensor([1,0.5,0.5], requires_grad=False, dtype=dtype, device=device)
W = torch.eye(3, requires_grad=True, dtype=dtype, device=device)
print(V1)
print(W)
O = W @ V1
print(O,O.norm())
optimizer = optim.Adam({W}, lr=1e-4)
    
i = 0
while True:
    i += 1
    optimizer.zero_grad()
    O = W @ V1
    loss = O.norm() 
    if i % 1000 == 0:
        print(i,loss)
    if(loss < 0.0012):
        break
    loss.backward()                                      
    optimizer.step()
print(i,W,O)
V2 = torch.tensor([0,1,0], requires_grad=False, dtype=dtype, device=device)
i = 0
while True:
    i += 1
    optimizer.zero_grad()
    O = W @ V2
    loss = O.norm() 
    if i % 1000 == 0:
        print(i,loss)
    if(loss < 0.0012):
        break
    loss.backward()                                      
    optimizer.step()
print(W@V1,W@V2)
print(W)

In [ ]:
ortho = Orthogonalizer(3)
V1 = torch.tensor([0.5,0,0.1], requires_grad=False, dtype=dtype, device=device).t()
V2 = torch.tensor([1,0,1], requires_grad=False, dtype=dtype, device=device).t()
V3 = torch.tensor([1,0.5,1], requires_grad=False, dtype=dtype, device=device).t()
print("n v1",ortho.novelty(V1))
print(ortho.O)
print("n v1",ortho.novelty(V1))
print("n v2",ortho.novelty(V2))
print("V1",V1)
print("add V1")
ortho.addBasis(V1)
print(ortho.O)
print("p v1",ortho.project(V1))
print("n v1",ortho.novelty(V1))
print("n v2",ortho.novelty(V2))
print("add V1")
ortho.addBasis(V1)
print(ortho.novelty(V1))
print("add V1")
ortho.addBasis(V1)
print(ortho.novelty(V1))
print("add V1")
ortho.addBasis(V1)
print(ortho.novelty(V1))
print("add V1")
ortho.addBasis(V1)
print(ortho.novelty(V1))
print("add V1")
ortho.addBasis(V1)
print("O",ortho.O)
print(ortho.novelty(V1))
print(ortho.novelty(V2))
print(ortho.novelty(V3))
print(ortho.project(V1))
print(ortho.project(V2))
print(ortho.project(V3))

ortho.addBasis(V2)
print(ortho.novelty(V1))
print(ortho.novelty(V2))
print(ortho.novelty(V3))
print(ortho.project(V1))
print(ortho.project(V2))
print(ortho.project(V3))

In [ ]:
ortho.addBasis(V1)
print(ortho.novelty(V1))
print(ortho.novelty(V2))
print(ortho.novelty(V3))
ortho.addBasis(V2)
print(ortho.novelty(V1))
print(ortho.novelty(V2))
print(ortho.novelty(V3))
ortho.addBasis(V3)
print(ortho.novelty(V1))
print(ortho.novelty(V2))
print(ortho.novelty(V3))
print(ortho.O)

In [ ]:
print(V1,V1.t())
print(ortho.O)
print(ortho.project(V1))
print(ortho.project(V2))
ortho.addBasis(V1)
print(ortho.O)
print(ortho.project(V1))
print(ortho.project(V2))
ortho.addBasis(V2)
print(ortho.O)
print(ortho.project(V1))
print(ortho.project(V2))
ortho.addBasis(V2)
print(ortho.O)
print(ortho.project(V1))
print(ortho.project(V2))

In [ ]:
ortho = Orthogonalizer(3)
V1 = torch.tensor([[0.5,0.25,0.25]], requires_grad=False, dtype=dtype, device=device).t()
print("V1.p",ortho.project(V1))
print("V1.n",ortho.novelty(V1))
print(ortho.O)
ortho.addBasis(V1)
print(ortho.O)
print("V1.p",ortho.project(V1))
print("V1.n",ortho.novelty(V1))
ortho.addBasis(V1)
print(ortho.O)
print("V1.p",ortho.project(V1))
print("V1.n",ortho.novelty(V1))
ortho.addBasis(V1)
print(ortho.O)
print("V1.p",ortho.project(V1))
print("V1.n",ortho.novelty(V1))
ortho.addBasis(V1)
print(ortho.O)
print("V1.p",ortho.project(V1))
print("V1.n",ortho.novelty(V1))

In [ ]:
V2 = torch.tensor([[1,3,4]], requires_grad=False, dtype=dtype, device=device).t()
print("x",ortho.novelty(V2))
ortho.addBasis(V2)
print(ortho.project(V2))
ortho.addBasis(V2)
print(ortho.project(V2))
ortho.addBasis(V2)
print(ortho.project(V2))
ortho.addBasis(V2)
print(ortho.project(V2))
print(ortho.project(V1))

##### gram-schmidt orthogonalization. Add a new basis one column at a time.
class Orthogonalizer:
    def __init__(self,size,thresh=1e-10):
        self.O = torch.eye(size, requires_grad=False, dtype=torch.float)
        self.O2 = torch.eye(size, requires_grad=False, dtype=torch.float)
        self.threshold = thresh
        return
    def novelty(self,X):
        return torch.mm(self.O,X)
    def project(self,X):
        return X - self.novelty(X)
    def addBasis(self,X):
        for x in X.t():
            a = self.novelty(x.unsqueeze(0).t())
            if (torch.abs(a).max() > self.threshold):
                self.O -= torch.div(torch.mm(a,a.t()),torch.norm(a).pow(2))
        return
    
#gram-schmidt orthogonalization. Add a new basis one column at a time.
class Orthogonalizer2:
    def __init__(self,size,thresh=1e-10):
        self.O = torch.eye(size, requires_grad=False, dtype=torch.float)
        self.O2 = torch.eye(size, requires_grad=False, dtype=torch.float)
        self.threshold = thresh
        return
    def novelty(self,X):
        return torch.mm(self.O,X)
    def project(self,X):
        return torch.div(torch.mm(self.O,X),self.O)
    def addBasis(self,X):
        for x in X.t():
            a = self.novelty(x.unsqueeze(0).t())
            if (torch.abs(a).max() > self.threshold):
                self.O -= torch.div(torch.mm(a.t(),a),torch.norm(a).pow(2))
        return
    
class Forgetful_Orthogonalizer(Orthogonalizer):
    def addBasis(self,X):
        #Reduce magnitude of prior orthoganlizations 
        super(Forgetful_Orthogonalizer, self).addBasis(X)
        self.O = torch.tanh(self.O - self.O2) + self.O2
        return
    
def compare(V,ortho,fortho):
    ortho.addBasis(V)
    fortho.addBasis(V)
    print('Value {}\tNovelty: {}\tF Novelty: {}\tProjection: {}\tF Projection {}'.format(
        V, ortho.novelty(V),fortho.novelty(V),ortho.project(V),fortho.project(V)))
def compare2(V,ortho):
    ortho.addBasis(V)
    Q,_ = np.linalg.qr(V)
    print('\tO {}\t Q P{}\tValue {}\tNovelty: {}\tProjection: {}'.format(
        ortho.O,Q,V, ortho.novelty(V),ortho.project(V),fortho.project(V)))

def make_householder(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(v, v)) * np.dot(v[:, None], v[None, :])
    return H
def householder_v(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)

    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)

    return v,tau
    
ortho = Orthogonalizer(3)
fortho = Forgetful_Orthogonalizer(3)
V1 = torch.tensor([[0.5,0.25,0.25]], requires_grad=False, dtype=torch.float).t()
print("x")
print(ortho.project(V1))
print(ortho.novelty(V1))
ortho.addBasis(V1)
print(ortho.project(V1))
ortho.addBasis(V1)
print(ortho.project(V1))
ortho.addBasis(V1)
print(ortho.project(V1))
ortho.addBasis(V1)
print(ortho.project(V1))

In [ ]:
V1 = torch.tensor([[0.5,0.25,0.25]], requires_grad=False, dtype=torch.float).t()
V2 = torch.tensor([[2,3,4]], requires_grad=False, dtype=torch.float).t()
Q,r = np.linalg.qr(V1)
print("Qr",np.linalg.qr(V1))
print("householder",householder_v(V1.numpy()))
# print("householder2",make_householder(V1.numpy()))
h,r2 = householder_v(V1.numpy())
Q = torch.tensor(Q)
print("Qt.Q",Q @ Q.t())
print("Q.Qt",torch.mm(Q,Q.t()))
print("Qt.Q",torch.mm(Q.t(),Q))
print(Q @ Q.t() @ V1)#Projection from Q
print(h @ h.T @ V1.numpy())#Projection from h
print(V1 - torch.mm(torch.mm(Q,Q.t()),V1))#Novelty from Q
print("")


Q,r = np.linalg.qr(V2)
Q = torch.tensor(Q)
print("Q",Q)
print("Q.Qt",torch.mm(Q,Q.t()))
print("Qt.Q",torch.mm(Q.t(),Q))
print(torch.mm(torch.mm(Q,Q.t()),V2))#Projection from Q
print(V2 - torch.mm(torch.mm(Q,Q.t()),V2))#Novelty from Q
print("")


print(torch.mm(torch.mm(Q,Q.t()),V1))#Projection from Q
print(V1 - torch.mm(torch.mm(Q,Q.t()),V1))#Novelty from Q
print("")

In [ ]:
B: tensor([[3., 1., 3., 3., 3., 3.],
        [3., 3., 6., 4., 4., 4.],
        [4., 3., 5., 5., 5., 5.]]), projection: tensor([[5.9071, 2.0330, 7.4282, 5.7807, 5.7807, 5.7807],
        [4.1150, 2.9704, 7.2674, 4.8306, 4.8306, 4.8306],
        [1.4824, 1.9224, 1.4342, 2.4399, 2.4399, 2.4399]])

def gram_schmidt(A):
    """Orthogonalize a set of vectors stored as the columns of matrix A."""
    # Get the number of vectors.
    n = A.shape[1]
    for j in range(n):
        # To orthogonalize the vector in column j with respect to the
        # previous vectors, subtract from it its projection onto the
        # each of the previous vectors.
        for k in range(j):
            A[:, j] -= np.dot(A[:, k], A[:, j]) * A[:, k]
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
    return A
A = np.array([[1.0, 1.0, 0.0], [1.0, 3.0, 1.0], [2.0, -1.0, 1.0]])
# print(gram_schmidt(A))

def gram_schmidt_columns(X):
    Q, R = np.linalg.qr(X)
    return Q,R
Q,R = gram_schmidt_columns(V4)
print("zz",V4.numpy().dot(R.T))
print(V4)
print('sss',gram_schmidt_columns(V4))
z = Orthogonalizer(3)
z.addBasis(V4)
print(Q.dot(Q.T).dot(V4))
print(V4.numpy()-Q.dot(Q.T).dot(V4))
print(z.novelty(V4))
print(z.project(V4))

In [45]:
eps = 1

#            certainty_factor = self.alpha/torch.exp(abs(self.loss))
s = [0,1,2,3,4,5]
print(np.mean(s))
for i in range(6):
    v = i /5
    X = torch.tensor([v], requires_grad=False, dtype=dtype, device=device)
    print(X,1./torch.exp(X*4/0.0001))

2.5
tensor([0.]) tensor([1.])
tensor([0.2000]) tensor([0.])
tensor([0.4000]) tensor([0.])
tensor([0.6000]) tensor([0.])
tensor([0.8000]) tensor([0.])
tensor([1.]) tensor([0.])
